In [4]:
from os import listdir
from os.path import isfile, join
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from gensim.similarities import WmdSimilarity
from gensim.summarization.summarizer import summarize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronnald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
with open('label_content', 'rb') as f:
    labelContent = pickle.load(f)

In [6]:
tagged_data = [TaggedDocument(words=word_tokenize(content.lower()), tags=[str(label)]) for label, content in labelContent.items()]

In [7]:
max_epochs = 10
vec_size = 100#neurons
alpha = 0.025
window = 3

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                window = window,binary=True)
  
model.build_vocab(tagged_data)



In [8]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    #model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [9]:
to_measure = {}
for i in range(0,len(tagged_data)):
    to_measure[i] =tagged_data[i][1][0]

In [11]:
from time import time
import numpy as np
start = time()

Jict = {}
distance =np.zeros(shape=(len(to_measure),len(to_measure)))

for i in range(0,len(to_measure)):
    Jict[to_measure[i]] = {}
    for j in range(0,len(to_measure)):
        rel = model.wv.wmdistance(document1=to_measure[i],document2=to_measure[j])
        Jict[to_measure[i]][to_measure[j]] = {'weight': rel}
        distance[i][j] = rel

print('Took %.2f seconds to run.' % (time() - start))

Took 426.52 seconds to run.


In [13]:
print(to_measure[1])
print(to_measure[66])

2018-2-350190
2018-1-302059


In [20]:
print(tagged_data[1][1])
print(tagged_data[66][1])

['2018-2-350190']
['2018-1-302059']


In [14]:
with open('label_sentence', 'rb') as f:
    labelSentence = pickle.load(f)

In [15]:
print(summarize(labelSentence["2018-2-350190"],ratio=0.2))

O brasileiro tem a necessidade de ampliar seu campo de protecao, tendo um duplo pertencimento de acordo com suas expectativas pessoais (experimentacao religiosa e pessoal) e aquilo que e imposto pela sociedade como, por exemplo, uma tradicao familiar.


In [16]:
print(summarize(labelSentence["2018-1-302059"],ratio=0.2))

Por isso, e importante que a religiao forneca um significado e sentido para as pessoas.
Ademais, Peter Berger fala sobre como as religioes tem potencial de situar e integrar as experiencias num significado, favorecendo um referencial importante para a construcao e manutencao de um ideal.
Alem disso, apresenta caracteristicas e desafios para o crescente pluralismo no campo religioso brasileiro como sua democratizacao, a liberdade religiosa, a mobilidade religiosa, a escolha religiosa.
Falamos sobre a educacao intercultural, isto e, aquela que orienta sobre os processos de reconhecimento do direito a diferenca e de luta contra todas as formas de discriminacao e desigualdade social, que promove o dialogo entre pessoas com pensamentos e culturas diferentes, e sobre o dialogo ecumenico, ou seja, dialogo entre as igrejas cristas.
Uma novidade para mim foi saber sobre a presenca de elementos do espiritualismo kardecista e do catolicismo popular na Umbanda e que e caracterizada por ser tipicam

In [20]:
with open('wmd', 'wb') as f:
        pickle.dump(distance, f)


In [33]:
with open('Jict', 'wb') as f:
        pickle.dump(Jict, f)

In [19]:
distance

array([[0.        , 2.34258449, 3.6922932 , ..., 2.26872713, 2.3650414 ,
        1.80312719],
       [2.34258449, 0.        , 3.65823327, ..., 2.0964097 , 2.28415459,
        1.78203718],
       [3.6922932 , 3.65823327, 0.        , ..., 2.53657278, 1.70487575,
        3.48318312],
       ...,
       [2.26872713, 2.0964097 , 2.53657278, ..., 0.        , 1.89452569,
        2.08347679],
       [2.3650414 , 2.28415459, 1.70487575, ..., 1.89452569, 0.        ,
        2.65196343],
       [1.80312719, 1.78203718, 3.48318312, ..., 2.08347679, 2.65196343,
        0.        ]])